# SEN1221 Statistical Analysis of Choice Behaviour 

## `Session Lab 02:`
## `The Mixed Logit model`

**Delft University of Technology**<br>
**Q2 2023**<br>
**Instructor:** Sander van Cranenburgh<br>
**TA:**  Gabriel Nova <br>

### `Instructions`

**Lab sessions aim to:**<br>
* Illustrate how models and theory discussed in the classroom work out in practice.
* Help you gather hands-on modelling and data analysis skills.


**Lab sessions are:**<br>
* Learning environments where you work with Python and get support from TA and fellow students.
* Not graded and do not have to be submitted.
* A good preparation for the graded partial exam.

### `Use of AI tools`
AI tools, such as ChatGPT and Co-pilot, are great tools to assist with programming. Moreover, in your later careers, you will work in a world where such tools are widely available. As such, we **encourage** you to use AI tools **effectively**. However, be careful not to overestimate the capacity of AI tools! AI tools cannot replace you: you still have to conceptualise the problem, dissect it and structure it to conduct proper analysis. We recommend being especially **reticent** with using AI tools for the more conceptual and reflection-oriented questions. <br>
Futhermore **be aware** that during the `partial exam`, you will not have access to these tools (since internet access will be restricted).

### `Workspace set-up`

**Option 1: Local environment**<br>
Uncomment the following cell if you are running this notebook on your local environment to install all dependencies on your Python version.

In [ ]:
#!pip install -r requirements.txt

**Option 2: Google Colab**<br>
Uncomment the following cell if you are running this notebook on Colab

In [ ]:
#!git clone https://github.com/SEN1221TUD/Q2_2023
#!mv "/content/Q2_2023/Lab sessions/lab_session_2/data" /content/data
#!mv "/content/Q2_2023/Lab sessions/lab_session_2/biogeme.toml" /content

#!pip install biogeme
#import os
#os.kill(os.getpid(), 9)

### `Application: Estimating the Value of Travel Time`

In this lab session, we will investigate the "Value of Travel Time" (VTT) distribution. The VTT of a traveller reflects the amount of money the traveller is **willing to pay** to reduce their travel time. The VTT is used to determine the benefits of new infrastructure projects. As travel time savings are the dominant and most salient benefits of new infrastructure, accurate inference of the distribution of the VTT is crucial for a rigorous underpinning of policy decisions. <br>

During this lab, we will apply Mixed Logit choice models. We aim to uncover how tastes for travel time and travel cost are distributed in the population. Most of the analyses in this lab session are carried out in the so-called willingness-to-pay space. Willingness-to-pay space facilitates the inference of the VTT distribution.<br>

For this study, we will use Stated Choice (SC) data (`Norway_VTT_2009.csv`) collected in 2009 to compute the Norwegian VTT. In this SC experiment, respondents faced nine choice tasks involving two alternatives and two attributes (travel cost and travel time). The data set consists of 5,832 participants, resulting in a total of 52,488 choice observations. The figure below shows one of the choice tasks (note that for the purposes of illustration we converted the currency unit (Kronor) into euros).

![SC](data/sc_experiment.png)

**`Learning objectives lab session 02`**

After completing the following exercises, you will be able to:
* Estimate Mixed Logit models that capture taste heterogeneity and panel effects
* Interpret the modelling outcomes of Mixed Logit models
* Specify utility models in Willingness-to-pay space
* Discuss the impact of the number of draws on the modelling outcomes



**`This lab consists of 5 parts and has 3 exercises`**

**Part 1**: Load and explore the data set

**Part 2**: Linear-additive RUM-MNL model

**Part 3**: Mixed Logit model for capturing taste heterogeneity 

**Part 4**: Willingness-to-Pay space 
- Exercise 1: "ML with log-normally distributed VTT"

**Part 5**: Panel Mixed Logit model

- Exercise 2: "Panel ML model with log-normally distributed VTT"

- Exercise 3: "Impact of the number of draws"



### `Import packages`

To begin, we will import all the libraries that we will use in this lab.

In [ ]:
# Biogeme
import biogeme.logging as blog
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable, bioDraws, log, MonteCarlo, exp, bioMultSum, exp

# General packages
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path
import toml
import time
from scipy.stats import norm, lognorm

# Pandas setting to show all columns when displaying a pandas dataframe
pd.set_option('display.max_columns', None)


## `1. Load and explore the data set` <br>

First, we load the data.

In [ ]:
# 1. Load the data set
data_path = Path('data/Norway_VTT_2009.csv')
df = pd.read_table(data_path, sep=',')

# Show descriptive statistics
round(df.describe(),2)

This data set contains the following variables:<br>

| Variable       | Description                                                  | Type    |
|---------------|-------------------------------------------------------------|---------|
| `RespID`        | Unique identifier for each survey response             | Integer |
| `CostL`         | Travel cost of left alternative [min]                  | Decimal |
| `CostR`         | Travel cost of right alternative [min]                 | Decimal |
| `TimeL`         | Travel time of left alternative [eur]                  | Decimal |
| `TimeR`         | Travel time of right alternative [eur]                 | Decimal |
| `Chosen`        | Indicates the alternative chosen       | Categorical |
| `Mode`          | Type of Transport mode               | Categorical    |
| `Gender`        | Gender of the respondent                                     | Categorical    |
| `AgeClass`      | Classification of the respondent's age                       | Categorical    |
| `IncClass`      | Classification of the respondent's income                    | Categorical    |
| `Purpose`       | Purpose of the trip                                     | Integer    |


Now, we take the following data preprocessing steps:

1. We keep only observations for the purpose 'long distance commute' and travel model 'car'.

2. We convert the unit of cost from Norwegian Krone to euros to ease interpretation.

In [ ]:
# 1. Keep only entries purpose == 5 (Long distance trips) & Mode == 1 (Car)
df = df.loc[(df['Purpose'] == 5) & (df['Mode'] == 1)]

In [ ]:
# 2. Convert the monetary unit to euros
NOK2euro_exchange_rate = 9
df[['CostL','CostR']] = df[['CostL','CostR']] .div(NOK2euro_exchange_rate)

## `2. Linear-additive RUM-MNL model`<br>
We first estimate a linear-aditive RUM-MNL model. This model serves as our **benchmark** to compare against. But, before we can do this, we need to create the Biogeme database object and specify the optimiser and logger settings. 

`Biogeme database`<br>
To estimate a model in Biogeme, we must create the data set as a Biogeme database object and the attributes as Biogeme variable objects.

In [ ]:
# Create Biogeme database object
biodata = db.Database('Norway2009VTT', df)

# Create Biogeme variable objects
CostL  = Variable('CostL')
CostR  = Variable('CostR')
TimeL  = Variable('TimeL')
TimeR  = Variable('TimeR')
Chosen = Variable('Chosen')	

`Biogeme optimiser and logging settings`<br>
The file `biogeme.toml` contains the settings for the optimiser. In this file, we set the number of draws for estimating Mixed Logit models to 50. We use a relatively low number to avoid long estimation times.<br>
Also, we invoke a so-called `logger` which enables us to see the progress during estimation.<br>

In [ ]:
# Change the number of draws in the .toml file. 
with open('biogeme.toml', 'r') as file:
    tomldata = toml.load(file)

# Modify the number of draws
tomldata['MonteCarlo']['number_of_draws'] = 50

# Write the modified data back to the .toml file
with open('biogeme.toml', 'w') as file:
    toml.dump(tomldata, file)

# Create a logger to monitor the estimation progress
# if logger does not exist create it, else use it
try:
    logger
except NameError:    
    logger = blog.get_screen_logger(level=blog.INFO)

`Create a linear-additive RUM-MNL model`<br>
Now, we have the biogeme database object and set the environment, we can estimate our first model.

In [ ]:
# Give the model a name
model_name = 'Benchmark MNL VTT model'

# Define model parameters
B_tt = Beta('B_tt', -0.1, None, None, 0)
B_tc = Beta('B_tc', -0.1, None, None, 0)

# Definition of the utility functions
VL = B_tt * TimeL + B_tc * CostL
VR = B_tt * TimeR + B_tc * CostR

# Associate utility functions with alternatives
V = {1: VL, 2: VR}     

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1} 

# Compute probability of the chosen alternative
prob = models.logit(V, av, Chosen)

# Take the log of the probability
logprob = log(prob)

# Create the Biogeme estimation object containing the data and the model
biogeme = bio.BIOGEME(biodata, logprob)

# Set reporting levels
biogeme.generate_pickle = False
biogeme.generate_html = False
biogeme.saveIterations = False
biogeme.modelName = model_name

# Compute the null loglikelihood for reporting
biogeme.calculateNullLoglikelihood(av)

# Estimate the model and print the results
results = biogeme.estimate()
print(results.short_summary())

# Get the results in a pandas table
beta_hat = results.getEstimatedParameters()
print(beta_hat)

`Compute the Value of Travel Time`<br><br>
The linear-additive RUM-MNL model allows for easy computation of the (mean) VTT. <br>

$VTT = {\frac{dV}{dtt}}/{\frac{dV}{dtc}}$<br><br>
$VTT = \frac{\beta_{tt}}{\beta_{tc}}$

To take the ratio, we access the estimated betas in the `beta_hat` dataframe that was created in the previous cell.

In [ ]:
# Compute the value of travel time and print the mean VTT
# We multiply by 60 to convert the value of travel time from minutes to hours
VTT_MNL = 60*(beta_hat.loc['B_tt']['Value']/beta_hat.loc['B_tc']['Value'])
print(f'Value of travel time MNL model:  €{VTT_MNL:.2f} per hour')

## `3. Mixed Logit model for capturing taste heterogeneity`

### `3.1 Theory`<br>
In the MNL model, we postulate that tastes (e.g., 𝛽_tc) are equal across people in the population. As such, the taste parameter of an MNL model represents the mean taste in the population. The Mixed Logit (ML) model resolves this limitation. It explicitly models taste heterogeneity by means of random variables.

Mathematically, the unconditional choice probability is given by:

$P_{ni} = \int_{\beta_n}    [P_{ni}|\beta_n] \cdot f(\beta_n|\sigma)d\beta_n$

As can be seen, the ML choice probability does not have a closed-form expression. Therefore, it needs to be approximated using simulation.<br>
To do that, we simulate the choice probabilities using a large number of draws `(R)` from the density function $f(β_n|\sigma)$. <br>
That is, we compute the conditional choice probability (which is a simple MNL) for each draw of $\beta_n^{r}$ with $r=1,..,R$, and then take the average across the draws to compute the unconditional choice probability.

<span style="text-decoration: overline;">P</span>$_{ni} = \frac{1}{R} \sum_{r=1}^R P_{ni}(β_n^r)$

Finally, we use the unconditional choice probabilities to compute the Log-Likelihood of the data given the model:

$LL(X,|\beta,\sigma)= \sum_{n=1}^N \sum_{j=1}^J y_{nj} \cdot ln($<span style="text-decoration: overline;">P</span>$_{nj})$

### `3.2 ML with normally distributed taste parameters`

To estimate an ML, model with normally distributed taste parameters, we must specify the random parameters (for all randomly distributed betas that we wish to estimate). To do this in Biogeme, we use the following code to construct the random parameter for $\beta_{tt}$:<br>

                B_tt_rnd = B_tt + sigma_tt * bioDraws('B_tt_rnd', 'NORMAL_HALTON2')

Note that apart from the random parameter for B_tt_rnd, the utility function is the same as under our linear-additive RUM-MNL benchmark model

`Model specification` <br>

Now, we will define random and nonrandom parameters, the utility functions, and their availabilities. <br>

In [ ]:
# Define the model parameters
B_tt = Beta('b_tt', -0.1, None, None, 0)
B_tc = Beta('b_tc', -0.1, None, None, 0)    
sigma_tt = Beta('sigma_tt', 1, None, None, 0)

# Construct the random taste parameter for beta_tt
B_tt_rnd = B_tt + sigma_tt * bioDraws('B_tt_rnd', 'NORMAL_HALTON2')

# Definition of the utility functions 
V_L = B_tt_rnd * TimeL + B_tc * CostL
V_R = B_tt_rnd * TimeR + B_tc * CostR   

# Create a dictionary to list the utility functions with the numbering of alternatives
V = {1: V_L, 2: V_R}
            
 # Create a dictionary to describe the availability conditions of each alternative
av = {1: 1, 2: 1} 

`Estimation through simulation` 

We use the Monte Carlo simulation to estimate the random parameter. 

In [ ]:
# Give the model a name
model_name = 'ML with normal distributed B_tt'

# The conditional probability of the chosen alternative is a logit
condProb = models.logit(V, av ,Chosen)

# The unconditional probability is obtained by simulation
uncondProb = MonteCarlo(condProb)

# The Log-likelihood is the log of the unconditional probability
LL = log(uncondProb)

# Create the Biogeme estimation object containing the data and the model
biogeme = bio.BIOGEME(biodata , LL)

# Set reporting levels
biogeme.generate_pickle = False
biogeme.generate_html = False
biogeme.saveIterations = False
biogeme.modelName = model_name

# Compute the null loglikelihood for reporting
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()
print(results.short_summary())

# Get the results in a pandas table
beta_hat = results.getEstimatedParameters()
print(beta_hat)

In [ ]:
# Compute the mean value of travel time
VTT_ML = 60*(beta_hat.loc['b_tt']['Value']/beta_hat.loc['b_tc']['Value'])
print(f'Value of travel time ML model:  €{VTT_ML:.2f} per hour')

### `3.3 Reflection`<br>
We do not also estimate $\beta_{tc}$ as a random parameter for two reasons.<br>
1. Jointly estimating $\beta_{tc-rnd}$ and $\beta_{tt-rnd}$ is tedious because of collinearity
2. When we compute the VTT we divide by $\beta_{tc}$. In case $\beta_{tc}$ would be **normally** distributed, we would divide by zero for some of the mass of the distribution. Dividing by zero leads to infinite VTT values. Hence, a normally distributed $\beta_{tc}$ is generally not a good idea.

## `4. Willingness-to-Pay space`

### `4.1. Theory`<br>

People are differently sensitive to cost. But, as discussed under 3.3, we cannot specify $\beta_{tc}$ as a randomly distributed parameter (or, to be more precise, not as one that uses a distribution which has support over the full domain, like a normal distribution). To circumvent this problem, we can cleverly re-parameterise our model. This reparametrisation involves a transformation from **`utility space`** to **`Willingness-to-Pay (WTP) space`**. This transformation allows us to estimate the VTT (distribution) directly, and simplifies the modelling. It works as follows:<br><br>


The utility specification in `utility space` is:

$V_1 = \beta_{tc} \cdot TC_1 + \beta_{tt} \cdot TT_1$<br>
$V_2 = \beta_{tc} \cdot TC_2 + \beta_{tt} \cdot TT_2$<br><br>

We factorise $\beta_{tc}$ in both utility functions. This gives us:

$V_1 = \beta_{tc} \cdot (TC_1 + (\frac{\beta_{tt}}{\beta_{tc}}) \cdot TT_1)$<br>
$V_2 = \beta_{tc} \cdot (TC_2 + (\frac{\beta_{tt}}{\beta_{tc}}) \cdot TT_2)$<br><br>

Noting that $VTT = \frac{\beta_{tt}}{\beta_{tc}}$, we obtain:

$V_1 = \beta_{tc} \cdot (TC_1 + VTT \cdot TT_1)$<br>
$V_2 = \beta_{tc} \cdot (TC_2 + VTT \cdot TT_2)$<br><br>

Hence, with this model, we can directly estimate the VTT (and $\beta_{tc}$). Therefore, this model is in the `Willingness-to-Pay space`. Let's see how this works out for a simple MNL model:

In [ ]:
# Give the model a name
model_name = 'Benchmark MNL in WTP space'

# Define model parameters
vtt  = Beta('vtt',     1, None, None, 0)
B_tc = Beta('B_tc', -0.1, None, None, 0)

# Definition of the utility functions
VL = B_tc * (CostL + vtt * TimeL)
VR = B_tc * (CostR + vtt * TimeR)

# Associate utility functions with alternatives
V = {1: VL, 2: VR}     

# Associate the availability conditions with the alternatives
av = {1: 1, 2: 1} 

# Compute probability of the chosen alternative
prob = models.logit(V, av, Chosen)

# Take the log of the probability
logprob = log(prob)

# Create the Biogeme estimation object containing the data and the model
biogeme = bio.BIOGEME(biodata, logprob)

# Set reporting levels
biogeme.generate_pickle = False
biogeme.generate_html = False
biogeme.saveIterations = False
biogeme.modelName = model_name

# Compute the null loglikelihood for reporting
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters and print the results
results = biogeme.estimate()
print(results.short_summary())

# Get the results in a pandas table
beta_hat = results.getEstimatedParameters()
print(beta_hat)

In [ ]:
# Compute the value of travel time
VTT_WTP_MNL = 60 * beta_hat.loc['vtt']['Value']
print(f'Value of travel time MNL model in WTP space:  €{VTT_WTP_MNL:.2f} per hour')

### `4.2 Reflection`<br>
Comparing these results with the benchmark MNL VTT model, we make two observations:
* We obtain `EXACTLY` the same VTT
* We obtain `EXACTLY` the same model fit 
* **We immediately obtain the VTT and the associated standard error!** 


### `4.3. ML in Willingness-to-Pay space`

`ML with normally distributed taste parameters`<br>
Now, let's see how WTP space enables us to directly estimate the VTT distribution in the context of the Mixed Logit model.

In [ ]:
# Give the model a name
model_name = 'ML WTP space with normally distributed vtt'

# Parameters definition enabling the construction of random parameters
vtt = Beta('vtt',    0.1, None, None, 0)
B_tc = Beta('b_tc', -0.1, None, None, 0)    
sigma_vtt = Beta('sigma_vtt', 1, None, None, 0)

# Construction of random parameters   
vtt_rnd = vtt + sigma_vtt * bioDraws('vtt_rnd', 'NORMAL_HALTON2')

# Definition of the utility functions 
V_L = B_tc * (CostL + vtt_rnd * TimeL)
V_R = B_tc * (CostR + vtt_rnd * TimeR)   

# Create a dictionary to list the utility functions with the numbering of alternatives
V = {1: V_L, 2: V_R}
            
 # Create a dictionary to describe the availability conditions of each alternative
av = {1: 1, 2: 1} 

In [ ]:
# The conditional probability of the chosen alternative is a logit
condProb = models.logit(V, av ,Chosen)

# The unconditional probability is obtained by simulation
uncondProb = MonteCarlo(condProb)

# The Log-likelihood is the log of the unconditional probability
LL = log(uncondProb)

# Create the Biogeme estimation object containing the data and the model
biogeme = bio.BIOGEME(biodata , LL)

# Set reporting levels
biogeme.generate_pickle = False
biogeme.generate_html = False
biogeme.saveIterations = False
biogeme.modelName = model_name


# Compute the null loglikelihood for reporting
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters and print the results
results = biogeme.estimate()
print(results.short_summary())

# Get the results in a pandas table
beta_hat = results.getEstimatedParameters()
print(beta_hat)

In [ ]:
# Compute the value of travel time
VTT_WTP_ML = 60*beta_hat.loc['vtt']['Value']
print(f'Value of travel time ML model in WTP space:  €{VTT_WTP_ML:.2f} per hour')

`Visualisation of the estimated VTT distribution`<br>
Visualisation of the estimated distribution helps to get a feeling for the shape of the distribution that we aim to recover. To do so, we use the estimated values for the mean (vtt) and standard deviation (sigma_vtt). <br>
Note that the visualisation distinguishes between values in the positive and negative domains.

In [ ]:
# Plot the estimated normal distribution

# Get the mean and standard deviation from the beta_hat table
mean_normal = beta_hat.loc['vtt']['Value']
std_dev_normal = beta_hat.loc['sigma_vtt']['Value']

# Create a vector of 100 points between +/- 4 standard deviations from the mean
x= np.linspace(mean_normal - 4*std_dev_normal, mean_normal + 4*std_dev_normal, 100)

# Plot the normal distribution in the positive domain. 
# Note that the x-axis is rescaled by 60 to convert from minutes to hours
# The scipy function "norm.pdf(x,mean, std_dev)" computes the probability density function of the normal distribution
plt.plot(x[x>0]*60, norm.pdf(x[x>0], mean_normal, std_dev_normal))

# Plot the normal distribution in the negative domain. 
plt.plot(x[x<0]*60, norm.pdf(x[x<0], mean_normal, std_dev_normal), color='red',linestyle='dashed')

# Add a vertical line to highlight the zero
plt.plot([0,0], [0,np.max(norm.pdf(x[x>0], mean_normal, std_dev_normal))], color='black',linestyle='dashed')

# Add labels and title
plt.xlabel('VTT [euro/hour]')
plt.ylabel('Probability Density')
plt.ylim([0,1])
plt.title(f'Distribution of the VTT')
plt.show()

### `4.4 Reflection`<br>
* Using a Mixed Logit model in the Willingness-to-Pay space, we are able to estimate the distribution of the VTT **directly** . Hence, this model allows jointly for unobserved heterogeneity in cost and time without running into problems caused by dividing by zero.  
* But, we see that a substantial part of the VTT distribution lies in the negative domain. This is behaviourally counterintuitive. In general, people prefer to arrive quicker at their destination rather than later. This is due to the assumption that the VTT is normally distributed. Perhaps this assumption needs revisiting...


## `Exercise 1: ML with log-normally distributed VTT`

Now, **you** will try the assumption that the VTT distribution in the population is **`log-normally distributed`** (as opposed to normally distributed).<br>

To do that, you must create a random vtt parameter with a log-normal VTT distribution. Draws from the log-normal distribution can be obtained from draws from the normal distribution as follows: <br>

            vtt_rnd = exp(mu + sigma  * bioDraws('vtt_rnd', 'NORMAL_HALTON2'))
            

Estimate this model and interpret its results.<br>

`Questions:`

`A` Compare the log-likelihood of the ML model with normal distribution and with log-normal distribution. Which model fits better?<br>

`B` What can you conclude from the `vtt` and `sigma` parameters?<br>

`C` Compute the mean of the estimated log-normal distribution, i.e. `mean_lognormal` (based on your estimated `mu` and `sigma`). <br>

**Hint**, take a look at [https://en.wikipedia.org/wiki/Log-normal_distribution](https://en.wikipedia.org/wiki/Log-normal_distribution) to see how this is done. Look for the formula for the mean in the right-hand side panel on the web page.

`D` Plot the shape of the log-normal distribution for the estimated mu and sigma. <br>

 **Hint**, you can use the  scipy function "lognorm". But lognorm is differently parameterised. That is, instead of using a location parameter `mu`, it requires a `scale` parameter. The scale is simply the exponent of the location parameter mu (i.e. scale = exp(mu)). <br>
    
Alternatively, you compute it directly from the formula yourself using the location `mu` and standard deviation $\sigma$:<br>
    ${\frac {1}{x\sigma {\sqrt {2\pi }}}}\ \exp \left(-{\frac {\left(\ln x-\mu \right)^{2}}{2\sigma ^{2}}}\right)$

In [ ]:
# your code

## `5. Panel Mixed Logit model`

Thus far, we have worked on the assumption that each choice observation is uncorrelated with all other choice observations. However, this data set contains multiple choices per respondent. In the ML modelling framework, we can also account for correlation in unobserved utility **across observations** of the same individual if we specify it as a panel ML model. In the panel ML model, the likelihood of the sequence of choices *t* = 1..*T* of an individual *n* is given by:  

$L_n(i_1,...,i_{T})(\beta_n|\sigma) = \int_{\beta_n}\Pi_{t=1}^T     P_{n}(i_t|\beta_n) f(\beta_n|\sigma)d\beta_n$

This likelihood does not have a closed-form expression. Therefore, as before, it needs to be approximated using simulation. Let's re-estimate the ML model assuming a normally distributed VTT distribution while accounting for panel structure. To do this, we first need to convert the data set into a so-called wide data format. In a wide format data set, each row contains all the choices belonging to an individual. Conveniently, Biogeme has a built-in function to do this (but, rather inconveniently, the names of the columns still need to be renamed).

### `5.1. Preparing a wide Biogeme database for estimating panel ML model`

In this cell we transform our data set into a wide format, and create a new Biogeme database object.

In [ ]:
# Tell Biogeme which variable is the identifier of the individuals
biodata.panel('RespID')

# Calculate the number of observations per individual
obs_per_ind = biodata.data['RespID'].value_counts().unique()[0]
print(f'Number of observations per individual: {obs_per_ind}')

# Use biogeme's "generateFlatPanelDataFrame to create a wide database in which each row corresponds to one individual
df_wide = biodata.generateFlatPanelDataframe(identical_columns=None)

# Rename the columns, such that they run from columnname_{0} to columnname_{n} 
renumbered_columns = {col: f'{col.split("_")[1]}_{int(col.split("_")[0])-1}' if len(col.split("_")) == 2 else col for col in df_wide.columns}

# Rename the columns using the dictionary
df_wide.rename(columns=renumbered_columns, inplace=True)

# Create Biogeme database object
biodata_wide = db.Database('Norway2009VTT_wide', df_wide)
biodata_wide.data.head()

### `5.2. Panel ML model with normally distributed VTT`

In [ ]:
# Give the model a name
model_name = 'Panel ML WTP space with normally distributed vtt'

# Parameters definition enabling the construction of random parameters
vtt       = Beta('vtt',       0.4, None, None, 0)
B_tc      = Beta('b_tc',     -0.4, None, None, 0)    
sigma_vtt = Beta('sigma_vtt ',  2, None, None, 0)

# Construction of random parameters   
vtt_rnd = vtt + sigma_vtt * bioDraws('vtt_rnd', 'NORMAL_HALTON2')

# Definition of the utility functions
# Note that we use list comprehension to create a list of utility functions for all observations of an individual 
V_L = [B_tc * (Variable(f'CostL_{q}') + vtt_rnd * Variable(f'TimeL_{q}')) for q in range(obs_per_ind)]
V_R = [B_tc * (Variable(f'CostR_{q}') + vtt_rnd * Variable(f'TimeR_{q}')) for q in range(obs_per_ind)]

# Create a dictionary to list the utility functions with the numbering of alternatives
# Note that we use list comprehension to create a list of dictionaries
V = [{1: V_L[q], 2: V_R[q]} for q in range(obs_per_ind)]
           
# Create a dictionary to describe the availability conditions of each alternative
av = {1:1, 2:1}

In [ ]:
# The conditional probability of the chosen alternative is a logit
condProb = [models.loglogit(V[q], av, Variable(f'Chosen_{q}')) for q in range(obs_per_ind)] 

# Take the product of the conditional probabilities
condprobIndiv = exp(bioMultSum(condProb))   # exp to convert from logP to P again

# The unconditional probability is obtained by simulation
uncondProb = MonteCarlo(condprobIndiv)

# The Log-likelihood is the log of the unconditional probability
LL = log(uncondProb)

# Create the Biogeme estimation object containing the data and the model
biogeme = bio.BIOGEME(biodata_wide , LL)

# Set reporting levels
biogeme.generate_pickle = False
biogeme.generate_html = False
biogeme.saveIterations = False
biogeme.modelName = model_name
                                
# Compute the null loglikelihood for reporting
# Note that we need to compute it manually, as biogeme does not do this for panel data
biogeme.nullLogLike = len(biodata_wide.data)*np.log(1/2)*obs_per_ind

# Estimate the parameters and print the results
results = biogeme.estimate()
print(results.short_summary())

# Get the results in a pandas table
beta_hat = results.getEstimatedParameters()
print(beta_hat)

In [ ]:
# Compute the value of travel time
VTT_WTP_ML_PANEL_normal = 60 * beta_hat.loc['vtt']['Value']
print(f'Value of travel time Panel ML model in WTP space:  €{VTT_WTP_ML_PANEL_normal:.2f} per hour')

## `Exercise 2: Panel ML with log-normally distributed VTT`

Now, **you** will estimate a ML model under the assumption that the VTT is log-normally distributed, while accounting for panel effects.<br>

To do so, copy the code from the Panel ML model in WTP space with normally distributed VTT, and create the log-normally distributed random parameter (as you have done in exercise 1).<br>  
Estimate this model and interpret the results.<br>

`Questions:`

`A` Compare the log-likelihood of the ML models with the log-normally distributed VTTs, which do and do not account for the panel effect. Which model fits better?<br>

`B` Compute the mean of the VTT for the Panel ML model with the log-normally distributed VTT and compare it with the non-panel model. Has it changed?<br>

`C`  i. Print the recovered mean VTTs of the models we have estimated below each other.<br>
* MNL model<br>
* ML model with Normal distribution in utility space<br>
* ML model with Normal distribution in wtp space<br>
* ML model with Log-normal in wtp space<br>
* Panel ML with Normal distribution in wtp space<br>
* Panel ML with Log-normal distribution in wtp space<br>                     

ii. Compare the VTTs of the models with a normal distribution and a log-normal distribution. Do you see a pattern? <br>

iii. What could explain this pattern?<br> 

In [ ]:
## Your code here

### `5.4. Impact of the number of draws on modelling outcomes`

## `Exercise 3: Impact of the number of draws` 

For all the Mixed Logit models that we have estimated, we have used 50 draws (see the beginning of this notebook). We choose a relatively low number of draws to avoid long estimation times.  <br>

Next, we analyse how sensitive the modelling outcomes are towards the number of draws. To do this, we have estimated a Panel Mixed Logit model using different numbers of draws, ranging from 33 to 2,000, and stored the results. <br>

The following plots show the results. 

![Draws](data/draws_vs_.png)

`Questions:`

`A` The left-hand side plot shows that the VTT estimate gets more stable with an increasing number of draws. Can you explain why the estimate gets more stable? 

`B` What number of draws do you deem sufficient for estimating this model? Explain your answer.

`C` The right-hand side plot shows a linear relation between the number of draws and the estimation time. Explain why a linear relation was to be expected.

`D` Suppose we estimate a model with *K* random parameters. Would the relation between the number of draws and estimation time still be linear? Explain your answer. 


`your anwsers here`<br>
A. ... 

B. ....
      
C. ... 
        
D. ...  

    


## END

In [ ]:
# Below is the code to create the plot 

# Create a dataframe to store the results
df_out = pd.DataFrame(columns=['num_draws','VTT', 'LL','elapsed_time'])

# Define the number of draws to be used for Monte-Carlo simulation
num_draws = list(range(33, 201, 33))

# Loop over the number of draws
for R in num_draws:
    
    # Start the timer
    start_time = time.time()

    # Give the model a name
    model_name = f'Panel ML WTP space with log-normally distributed vtt with {R} draws'

    # Change the number of draws in the .toml file
    with open('biogeme.toml', 'r') as file:
        data = toml.load(file)

    # Modify the value
    data['MonteCarlo']['number_of_draws'] = R

    # Write the modified data back to the .toml file
    with open('biogeme.toml', 'w') as file:
        toml.dump(data, file)

    # Parameters definition enabling the construction of random parameters
    vtt         = Beta('vtt',       0.4, None, None, 0)
    B_tc        = Beta('b_tc',     -0.4, None, None, 0)    
    sigma_vtt   = Beta('sigma_vtt',   2, None, None, 0)

    # Construction of random parameters   
    vtt_rnd = exp(vtt + sigma_vtt * bioDraws('vtt_rnd', 'NORMAL_HALTON2'))

    # Definition of the utility functions 
    V_L = [B_tc * (Variable(f'CostL_{q}') + vtt_rnd * Variable(f'TimeL_{q}')) for q in range(9)]
    V_R = [B_tc * (Variable(f'CostR_{q}') + vtt_rnd * Variable(f'TimeR_{q}')) for q in range(9)]

    # Create a dictionary to list the utility functions with the numbering of alternatives
    V = [{1: V_L[q], 2: V_R[q]} for q in range(9)]
            
    # Create a dictionary to describe the availability conditions of each alternative
    av = {1:1, 2:1}

    # The conditional probability of the chosen alternative is a logit
    condProb = [models.loglogit(V[q], av, Variable(f'Chosen_{q}')) for q in range(9)] 

    # Take the product of the conditional probabilities
    condprobIndiv = exp(bioMultSum(condProb))   # exp to convert from logP to P again

    # The unconditional probability is obtained by simulation
    uncondProb = MonteCarlo(condprobIndiv)

    # The Log-likelihood is the log of the unconditional probability
    LL = log(uncondProb)

    # Create the Biogeme estimation object containing the data and the model
    biogeme = bio.BIOGEME(biodata_wide , LL)
    
    # Set reporting levels
    biogeme.generate_pickle = False
    biogeme.generate_html = False
    biogeme.saveIterations = False
    biogeme.modelName = model_name
                                    
    # Compute the null loglikelihood for reporting
    biogeme.nullLogLike = len(biodata_wide.data)*np.log(1/2)*9

    # Estimate the parameters
    results = biogeme.estimate()
    # print(results.short_summary())

    # Get the results in a pandas table
    beta_hat = results.getEstimatedParameters()
    # print(beta_hat)

    # End the timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Elapsed time: {elapsed_time:.2f} seconds\n\n')

    # Compute the mean value of travel time
    mu = beta_hat.loc['vtt']['Value']
    sigma = beta_hat.loc['sigma_vtt']['Value'] 
    mean_lognormal_panel = np.exp(mu + np.square(sigma)/2) * 60
    
    # Add the results to the dataframe
    df_R = pd.DataFrame({'num_draws': [R], 'VTT': [mean_lognormal_panel], 'LL': [results.getGeneralStatistics()['Final log likelihood'][0]], 'elapsed_time': [elapsed_time]})
    df_out = pd.concat([df_out, df_R])

# Show the results
df_out

In [ ]:
# Plot the results in a figure
fig, ax = plt.subplots(1,3, figsize=(15,5), sharex=True)
fig.tight_layout(w_pad=3)

ax[0].plot(df_out['num_draws'], df_out['VTT'], marker='.')
ax[0].set_xlabel('Number of draws')
ax[0].set_ylabel('VTT [euro/hour]')
ax[0].set_title('VTT')

ax[1].plot(df_out['num_draws'], df_out['LL'], marker='.')
ax[1].set_xlabel('Number of draws')
ax[1].set_ylabel('Log-likelihood')
ax[1].set_title('Log-likelihood')

ax[2].plot(df_out['num_draws'], df_out['elapsed_time'], marker='.')
ax[2].set_xlabel('Number of draws')
ax[2].set_ylabel('Elapsed time [s]')
ax[2].set_title('Elapsed time')

plt.show()